In [2]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras import optimizers
from keras.optimizers import Adam

In [3]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()
train_x = x_train.reshape(60000, 784) / 255
val_x = x_test.reshape(10000, 784) / 255

autoencoder = Sequential()
autoencoder.add(Dense(512,  activation='elu', input_shape=(784,)))
autoencoder.add(Dense(128,  activation='elu'))
autoencoder.add(Dense(10,    activation='linear', name="bottleneck"))
autoencoder.add(Dense(128,  activation='elu'))
autoencoder.add(Dense(512,  activation='elu'))
autoencoder.add(Dense(784,  activation='sigmoid'))
autoencoder.compile(loss='mean_squared_error', optimizer = Adam())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:
trained_model = autoencoder.fit(train_x, train_x, batch_size=1024, epochs=10, verbose=1, validation_data=(val_x, val_x))


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 51us/step - loss: 0.0793 - val_loss: 0.0514
Epoch 2/10
60000/60000 [==============================] - 3s 48us/step - loss: 0.0447 - val_loss: 0.0380
Epoch 3/10
60000/60000 [==============================] - 3s 48us/step - loss: 0.0347 - val_loss: 0.0309
Epoch 4/10
60000/60000 [==============================] - 3s 48us/step - loss: 0.0293 - val_loss: 0.0272
Epoch 5/10
60000/60000 [==============================] - 3s 48us/step - loss: 0.0262 - val_loss: 0.0245
Epoch 6/10
60000/60000 [==============================] - 3s 49us/step - loss: 0.0240 - val_loss: 0.0229
Epoch 7/10
60000/60000 [==============================] - 3s 49us/step - loss: 0.0227 - val_loss: 0.0218
Epoch 8/10
60000/60000 [==============================] - 3s 48us/step - loss: 0.0216 - val_loss: 0.0208
Epoch 9/10
60000/60000 [==============================] - 3s 49us/step - loss: 0.0207 - val_loss: 0.0199
Epoc

In [4]:
encoder = Model(autoencoder.input, autoencoder.get_layer('bottleneck').output)
encoded_data = encoder.predict(train_x)  # bottleneck representation
decoded_output = autoencoder.predict(train_x)        # reconstruction
encoding_dim = 10

# return the decoder
encoded_input = Input(shape=(encoding_dim,))
decoder = autoencoder.layers[-3](encoded_input)
decoder = autoencoder.layers[-2](decoder)
decoder = autoencoder.layers[-1](decoder)
decoder = Model(encoded_input, decoder)

In [5]:
%matplotlib inline
from keras.preprocessing import image
# if the img.png is not one of the MNIST dataset that the model was trained on, the error will be very high.
img = image.load_img("./cow.jpg", target_size=(28, 28), color_mode = "grayscale")
input_img = image.img_to_array(img)
inputs = input_img.reshape(1,784)/ 255
target_data = autoencoder.predict(inputs)
dist = np.linalg.norm(inputs - target_data, axis=-1)
print(dist)

[9.958763]


In [6]:
inputs = train_x[0].reshape((1,784))
target_data = autoencoder.predict(inputs)
dist = np.linalg.norm(inputs - target_data, axis=-1)
print(dist)

[5.51066576]
